In [6]:
import pandas as pd
from meteostat import Point, Daily
from datetime import datetime, timedelta
import requests
import ssl

ssl._create_default_https_context = ssl._create_unverified_context



In [8]:


def get_weather_data(lat, lon, start_date, end_date):
    location = Point(lat, lon)
    data = Daily(location, start_date, end_date)
    data = data.fetch()
    return data

# Load your dataset
nb_df = pd.read_csv('../data/processed/nb_cleaned.csv')



In [9]:
nb_df[['Trip Origin Lat', 'Trip Origin Lng']] = nb_df['Trip Origin'].str.split(',', expand=True).astype(float)
nb_df[['Trip Destination Lat', 'Trip Destination Lng']] = nb_df['Trip Destination'].str.split(',', expand=True).astype(float)

nb_df['Trip Start Time'] = pd.to_datetime(nb_df['Trip Start Time'])
nb_df['Trip End Time'] = pd.to_datetime(nb_df['Trip End Time'])



In [11]:
weather_conditions = []
nb_df= nb_df.head(100)
for index, row in nb_df.iterrows():
    lat, lon = row['Trip Origin Lat'], row['Trip Origin Lng']
    start_date = row['Trip Start Time']
    end_date = start_date + timedelta(days=1)  
    
    weather_data = get_weather_data(lat, lon, start_date, end_date)
    
    if not weather_data.empty:
        rain = weather_data['prcp'].iloc[0] > 0  
        weather_conditions.append('Rain' if rain else 'No Rain')
    else:
        weather_conditions.append('No Data')

nb_df['Weather Condition'] = weather_conditions



In [13]:
nb_df.to_csv('../data/processed/nb_with_weather_conditions.csv', index=False)

print(nb_df.head(100))

    Unnamed: 0  Trip ID                           Trip Origin  \
0            0   391996    6.508813001668548,3.37740316890347   
1            1   391997                   6.4316714,3.4555375   
2            2   391998           6.631679399999999,3.3388976   
3            3   391999           6.572757200000001,3.3677082   
4            4   392001                   6.6010417,3.2766339   
..         ...      ...                                   ...   
95          95   392195   6.492950899999999,3.380802399999999   
96          96   392198  6.601994249890971,3.3489699769657477   
97          97   392199                   6.5058143,3.3883071   
98          98   392200                    6.4357433,3.438317   
99          99   392203                   6.5715416,3.4973193   

                        Trip Destination     Trip Start Time  \
0            6.650969799999999,3.3450307 2021-07-01 07:28:04   
1        6.4280814653326,3.4721885847586 2021-07-01 06:38:04   
2            6.508324099999